In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from statsmodels.nonparametric.smoothers_lowess import lowess
from scipy import signal, stats 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA

Mounted at /content/drive


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# Reading the CSV file
adult_ankle = pd.read_csv('/Data/GroupData/adult_ankle.csv')
young_ankle = pd.read_csv('/Data/GroupData/young_ankle.csv')
adult_hand = pd.read_csv('/Data/GroupData/adult_hand.csv')
young_hand = pd.read_csv('/Data/GroupData/young_hand.csv')
adult_pocket = pd.read_csv('/Data/GroupData/adult_pocket.csv')
young_pocket = pd.read_csv('/Data/GroupData/young_pocket.csv')

all_data = adult_ankle.append(young_ankle)
all_data = all_data.append(adult_hand)
all_data = all_data.append(young_hand)
all_data = all_data.append(adult_pocket)
all_data = all_data.append(young_pocket)

# generating features

all_data = all_data.reset_index()
all_data['lowess_x'] = lowess(endog=all_data['AccCleanX'], exog=all_data['index'], frac=0.004)[:,1]
all_data['lowess_y'] = lowess(endog=all_data['AccCleanY'], exog=all_data['index'], frac=0.004)[:,1]
all_data['lowess_z'] = lowess(endog=all_data['AccCleanZ'], exog=all_data['index'], frac=0.004)[:,1]

inst_acc = np.zeros(len(all_data['AccNet'])) 
for i in range(0,len(all_data['AccNet'])):
  if i==0:
    inst_acc[0] = all_data['AccNet'].iloc[0]
  else:
    inst_acc[i] = all_data['AccNet'].iloc[i] - all_data['AccNet'].iloc[i-1]
all_data['AccInst'] = inst_acc


#putting features/labels into x and y
X = all_data[['AccInst','AccNet','AccCleanX','AccCleanY','AccCleanZ','lowess_x','lowess_y','lowess_z']]
y_age = all_data['Label'].str[:1]
y_pos = all_data['Label'].str[1:]
y = all_data['Label']

0       a
1       a
2       a
3       a
4       a
       ..
5227    p
5228    p
5229    p
5230    p
5231    p
Name: Label, Length: 5232, dtype: object

In [ ]:
#train different models

X_train, X_valid, y_train, y_valid = train_test_split(X, y_pos)

naive_bayes = make_pipeline(
    StandardScaler(),
    GaussianNB()
)

KNN = make_pipeline(
    StandardScaler(),
    KNeighborsClassifier(n_neighbors=5)
)


decision_tree = make_pipeline(
    StandardScaler(),
    DecisionTreeClassifier(max_depth=8)
)

random_forest = make_pipeline(
    StandardScaler(),
    RandomForestClassifier(n_estimators=100, max_depth=8, min_samples_leaf=5)
)

gradient_boost = make_pipeline(
    StandardScaler(),
    GradientBoostingClassifier(n_estimators=50, max_depth=8, min_samples_leaf=5)
)

neural_network = make_pipeline(
    StandardScaler(),
    MLPClassifier(solver='lbfgs', hidden_layer_sizes=(8,6,4), activation='logistic', max_iter=50000)
)

naive_bayes.fit(X_train,y_train)
KNN.fit(X_train,y_train)
decision_tree.fit(X_train,y_train)
random_forest.fit(X_train,y_train)
gradient_boost.fit(X_train,y_train)
neural_network.fit(X_train,y_train)

# print results

print('Naive Bayes Score: ', naive_bayes.score(X_valid, y_valid))
print('KNN Score: ', KNN.score(X_valid, y_valid))
print('Decision Tree Score: ', decision_tree.score(X_valid, y_valid))
print('Random Forest Score: ', random_forest.score(X_valid, y_valid))
print('Gradient Boosting Score: ', gradient_boost.score(X_valid, y_valid))
print('Neural Network Score: ', neural_network.score(X_valid, y_valid))


Naive Bayes Score:  0.9923547400611621
KNN Score:  0.996177370030581
Decision Tree Score:  0.9969418960244648
Random Forest Score:  0.9969418960244648
Gradient Boosting Score:  0.9977064220183486
Neural Network Score:  0.9946483180428135
